In [9]:
import sys
lines = ["4","2","3","31","32"]
# for line in sys.stdin:
#   lines.append(line)

for i in lines[1:]:
  inta = int(i)
  num = 0
  
  for j in range(70):
      if 2**j >= inta:
          num = j
          print(num)
          break
  

1
2
5
5


In [27]:
from itertools import permutations
lines = ["ffdcbgadc", "dbc"]
# for line in sys.stdin:
#    lines.append(line)
def contains_permutation(A, B):
    if len(B) > len(A):
        return False
    
    def count_chars(string):
        char_count = {}
        for char in string:
            if char in char_count:
                char_count[char] += 1
            else:
                char_count[char] = 1
        return char_count
    
    target_count = count_chars(B)
    window_count = {}
    
    for i in range(len(A)):
        if A[i] in window_count:
            window_count[A[i]] += 1
        else:
            window_count[A[i]] = 1
        
        if i >= len(B):
            left_char = A[i - len(B)]
            if window_count[left_char] == 1:
                del window_count[left_char]
            else:
                window_count[left_char] -= 1
        
        if window_count == target_count:
            print("true")
            return
    print("false")
    return
contains_permutation(lines[0],lines[1])

true


In [57]:
import sys
import ipaddress
lines = ["2",
         "SNAT --sip 192.168.0.0/24 --to 110.10.0.1",
         "DNAT --sip 192.168.1.0/24 --dip 110.10.0.1 --to 110.10.0.254:9999",
         "2",
         "192.168.0.1:50000-110.10.0.1:443",
         "192.168.1.1:50000-110.10.0.1:443"]
    
# 定义地址转换规则类
class AddressRule:
    def __init__(self, rule_type, sip=None, sport=None, dip=None, dport=None, to_ip=None, to_port=None):
        self.rule_type = rule_type
        self.sip = sip
        self.sport = sport
        self.dip = dip
        self.dport = dport
        self.to_ip = to_ip
        self.to_port = to_port

    # 判断规则是否匹配给定的IP报文
    def matches(self, packet):
        if self.sip and not packet.source_ip in self.sip:
            return False
        if self.sport and packet.source_port != self.sport:
            return False
        if self.dip and not packet.dest_ip in self.dip:
            return False
        if self.dport and packet.dest_port != self.dport:
            return False
        return True

# 定义IP报文类
class IPPacket:
    def __init__(self, source_ip, source_port, dest_ip, dest_port):
        self.source_ip = source_ip
        self.source_port = source_port
        self.dest_ip = dest_ip
        self.dest_port = dest_port

# 解析地址转换规则
def parse_rule(rule_str):
    parts = rule_str.split()
    #print(parts)
    rule_type = parts[0]
    sip, sport, dip, dport = None, None, None, None
    to_ip, to_port = None, None

    for i in range(1, len(parts)):
        if parts[i] == "--sip":
            sip = ipaddress.ip_network(parts[i + 1])
        elif parts[i] == "--sport":
            sport = int(parts[i + 1])
        elif parts[i] == "--dip":
            dip = ipaddress.ip_network(parts[i + 1])
        elif parts[i] == "--dport":
            dport = int(parts[i + 1])
        elif parts[i] == "--to":
            to_parts = parts[i + 1].split(':')
            to_ip = ipaddress.ip_address(to_parts[0])
            
            if len(to_parts) > 1:
                to_port = int(to_parts[1])
            
            #print(to_port)

    return AddressRule(rule_type, sip, sport, dip, dport, to_ip, to_port)

# 解析IP报文
def parse_packet(packet_str):
    source, dest = packet_str.split('-')
    source_ip, source_port = source.split(':')
    dest_ip, dest_port = dest.split(':')
    return IPPacket(ipaddress.ip_address(source_ip), int(source_port), ipaddress.ip_address(dest_ip), int(dest_port))

# 应用地址转换规则到IP报文
def apply_rule(rule, packet):
    if rule.rule_type == "SNAT":
        if rule.to_ip:
            packet.source_ip = rule.to_ip
        if rule.to_port:
            packet.source_port = rule.to_port
    elif rule.rule_type == "DNAT":
        if rule.to_ip:
            packet.dest_ip = rule.to_ip
        if rule.to_port:
            packet.dest_port = rule.to_port
    return packet

# 主函数
def main():
    # 解析地址转换规则
    num_rules = int(lines[0])
    rules = []
    for linea in lines[1:num_rules+1]:
        rule_str = linea
        rule = parse_rule(rule_str)
        rules.append(rule)

    # 解析IP报文
    num_packets = int(lines[num_rules+1])
    packets = []
    for linea in lines[num_packets+2:]:
        packet_str = linea
        packet = parse_packet(packet_str)
        packets.append(packet)

    # 应用地址转换规则到IP报文
    for packet in packets:
        for rule in rules:
            if rule.matches(packet):
                packet = apply_rule(rule, packet)
        print(f"{packet.source_ip}:{packet.source_port}-{packet.dest_ip}:{packet.dest_port}")

if __name__ == "__main__":
    main()

110.10.0.1:50000-110.10.0.1:443
192.168.1.1:50000-110.10.0.254:9999


In [81]:

lines = ["7 0 1 2 2 4 4 5", "4"]

    
array2 = lines[0].split()
array = [int(i) for i in array2]
array.sort()
print(array)

find = -1
for i in range(len(array)):
    if int(array[i]) == int(lines[1]):
        find = i+1
        break
print(find)

[0, 1, 2, 2, 4, 4, 5, 7]
5


In [83]:
lines = ["5", "5 5 5 1 3"]
def min_water_drops(n, heights):
    drops = 0
    
    for i in range(n):
        if heights[i] < 6:
            if i > 0 and heights[i-1] < 6:
                # 检查左边是否有满水的水滴，有则戳破并发射水滴
                drops += 6 - heights[i]
                heights[i] = 6
            if i < n - 1 and heights[i+1] < 6:
                # 检查右边是否有满水的水滴，有则戳破并发射水滴
                drops += 6 - heights[i]
                heights[i] = 6
                
    return drops

# Read input
n = int(lines[0])
m = list(map(int, lines[1].split()))

# Calculate and print the minimum number of drops needed
print(min_water_drops(n, m))

25
